In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from langchain_text_splitters import RecursiveJsonSplitter

In [2]:
MODEL = "gpt-4o-mini"
json_path = "C:/Users/aalperen.arda/Documents/GitHub/LLM-Biography-Analysis/main/structured_bios_new_english_fixed.json"

In [3]:
documents = []

with open(json_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

for person in json_data:
    content = json.dumps(person, ensure_ascii=False)
    doc = Document(page_content=content, metadata={"ad": person["ad"]})
    documents.append(doc)

In [4]:
'''text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"{len(chunks)} adet chunk oluşturuldu.")'''

# chunk oluşturmaya gerek yok, zaten tüm doc elementler yeterince küçük

'text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)\nchunks = text_splitter.split_documents(documents)\n\nprint(f"{len(chunks)} adet chunk oluşturuldu.")'

In [5]:
embeddings = OpenAIEmbeddings()
persist_dir = "C:/Users/aalperen.arda/Documents/GitHub/LLM-Biography-Analysis/main/chroma_vector_store"

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_dir
)

In [6]:
data = vectorstore._collection.get(include=["embeddings", "documents", "metadatas"])

embeddings = data["embeddings"]
texts = data["documents"]
names = [m.get("ad", f"doc_{i}") for i, m in enumerate(data["metadatas"])]

In [7]:
X = np.array(embeddings)

cos_sim_matrix = cosine_similarity(X)

k_word = 20
pairs = set()

for i in range(len(cos_sim_matrix)):
    top_indices = np.argsort(cos_sim_matrix[i])[::-1][1:k_word + 1]
    
    for j in top_indices:
        pair = tuple(sorted((i, j)))
        pairs.add(pair) 

print(f"{len(pairs)} benzersiz çift eşleştirmesi üretildi.")

3621 benzersiz çift eşleştirmesi üretildi.


In [8]:
system_prompt = """Sen biyografik metinler arasında ilişki kurmakta uzmanlaşmış bir yapay zekasın. Görevin, iki kişinin detaylı biyografilerine dayanarak aralarındaki olası ilişkileri analiz etmektir. İlişkiler, sadece somut ortaklıklarla değil, dolaylı ipuçları ve örtük bağlantılarla da kurulabilir.

Dikkat etmen gereken bazı detaylar şunlardır:
- Aynı okulda okumuş olabilirler ama yılları tutmuyorsa bu durumu ilişki olarak belirtme.
- Aynı kurumda çalışmış olsalar bile yılları örtüşmüyorsa MESLEKTAŞ sayılmaz.
- Hobi benzerlikleri varsa bu sadece İLGİ_ALANI_ORTAKLIĞI olarak not edilebilir.
- Eş veya çocuk isimlerinin benzerliği yalnızca dolaylı bir ilişki gösterebilir.
- Eğer aralarında herhangi bir anlamlı ilişki kurulamıyorsa, sadece "YOK" yaz.

Cevabın yalnızca aşağıdaki ilişki etiketlerinden oluşmalı ve virgülle ayrılmalı."""

In [9]:
client = OpenAI()
relationship_results = []

for i, j in tqdm(pairs, desc="LLM ile ilişki çıkarılıyor"):
    user_prompt = f"""
Aşağıda iki kişinin detaylı biyografileri bulunmaktadır. Lütfen bu biyografilere dayanarak, aralarında hangi ilişkiler olabilir analiz et.

Yalnızca aşağıdaki seçeneklerden uygun olanları belirt:

- AYNI_MEMLEKETTEN
- ILKOKUL_ARKADASI
- LISE_ARKADASI
- UNIVERSITE_ARKADASI
- YUKSEK_LISANS_ARKADASI
- DOKTORA_ARKADASI
- MESLEKTAS
- AYNI_KURUMDA_CALISMIS
- GIRISIM_ORTAGI
- AKADEMİK_CALISMA_ORTAKLIGI
- ILGI_ALANI_ORTAKLIGI
- AILE_BAGI
- YOK

Kurallar:
- Yalnızca ilişki adlarını yaz, açıklama yapma.
- Yıllara dikkat et, ilişkiler zaman örtüşmesine göre geçerli sayılır.
- Eş veya çocuk isim benzerlikleri doğrudan ilişki anlamına gelmez, ancak aile bağı olarak değerlendirilebilir.

Kişi 1:
{texts[i]}

Kişi 2:
{texts[j]}
"""
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )

        result = response.choices[0].message.content.strip().upper()
        relations = [r.strip() for r in result.split(",") if r and r != "YOK"]

        for rel in relations:
            relationship_results.append({
                "source": names[i],
                "target": names[j],
                "relation": rel
            })

    except Exception as e:
        print(f"HATA ({names[i]} ↔ {names[j]}):", e)

LLM ile ilişki çıkarılıyor:  16%|█▋        | 589/3621 [35:28<3:02:38,  3.61s/it]   


KeyboardInterrupt: 

In [ ]:
output_path = os.path.join(persist_dir, "llm_relationships.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(relationship_results, f, ensure_ascii=False, indent=2)

print(f"{len(relationship_results)} adet ilişki çıkarıldı ve kaydedildi → {output_path}")

In [ ]:
## RELATION kısmına açıklama eklemeyi dene